In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing  import StandardScaler,LabelEncoder
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet,SGDRegressor,BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.metrics import r2_score,root_mean_squared_error,mean_absolute_error
import joblib

import warnings
warnings.filterwarnings("ignore")

In [76]:
df = pd.read_csv('/Users/saptarshipal/DATA/used_car_price_prediction/data/vehicles.csv')
df1 = df.copy()
print(df1.shape)
print(df1.columns)


(426880, 26)
Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')


In [77]:
df1.head()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
0,7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
1,7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
2,7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
3,7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
4,7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


In [78]:
df1.drop(columns = ['id','url','region_url','VIN','image_url','description','lat','long'],axis = 1,inplace =True)

In [79]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        426880 non-null  object 
 1   price         426880 non-null  int64  
 2   year          425675 non-null  float64
 3   manufacturer  409234 non-null  object 
 4   model         421603 non-null  object 
 5   condition     252776 non-null  object 
 6   cylinders     249202 non-null  object 
 7   fuel          423867 non-null  object 
 8   odometer      422480 non-null  float64
 9   title_status  418638 non-null  object 
 10  transmission  424324 non-null  object 
 11  drive         296313 non-null  object 
 12  size          120519 non-null  object 
 13  type          334022 non-null  object 
 14  paint_color   296677 non-null  object 
 15  county        0 non-null       float64
 16  state         426880 non-null  object 
 17  posting_date  426812 non-null  object 
dtypes: f

In [80]:
df1.isnull().sum()

region               0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
drive           130567
size            306361
type             92858
paint_color     130203
county          426880
state                0
posting_date        68
dtype: int64

In [81]:
null_columns = df1.columns[df1.isnull().mean()>0.4]
df1.drop(columns = null_columns,axis =1,inplace=True)

In [82]:
df1.isna().mean()

region          0.000000
price           0.000000
year            0.002823
manufacturer    0.041337
model           0.012362
fuel            0.007058
odometer        0.010307
title_status    0.019308
transmission    0.005988
drive           0.305863
type            0.217527
paint_color     0.305011
state           0.000000
posting_date    0.000159
dtype: float64

In [83]:
df1.drop(df1[df1['price'] == 0].index,axis=0,inplace=True)
df1.shape

(393985, 14)

In [84]:
cat_cols = df1.columns[df1.dtypes == 'object']
num_cols = df1.columns[df1.dtypes != 'object']
print(f'The categorical columns are {cat_cols}.\n\n')
print(f'The numerical columns are {num_cols}.\n\n')

The categorical columns are Index(['region', 'manufacturer', 'model', 'fuel', 'title_status',
       'transmission', 'drive', 'type', 'paint_color', 'state',
       'posting_date'],
      dtype='object').


The numerical columns are Index(['price', 'year', 'odometer'], dtype='object').




In [85]:
df1[cat_cols].isna().sum()

region               0
manufacturer     16185
model             4701
fuel              2594
title_status      7734
transmission      1823
drive           120254
type             85932
paint_color     117149
state                0
posting_date        62
dtype: int64

In [86]:
df1[cat_cols].columns[df1[cat_cols].isna().sum().sort_values(ascending=False)>10000]

Index(['region', 'manufacturer', 'model', 'fuel'], dtype='object')

In [87]:
df1.drop(columns = ['region', 'manufacturer', 'model', 'fuel'],axis =1, inplace=True)

In [88]:
df1[num_cols].isnull().sum().sort_values(ascending=False)

odometer    2290
year        1173
price          0
dtype: int64

# Imputation

In [89]:
df1.isnull().sum()

price                0
year              1173
odometer          2290
title_status      7734
transmission      1823
drive           120254
type             85932
paint_color     117149
state                0
posting_date        62
dtype: int64

In [90]:
df1.title_status.value_counts()

title_status
clean         372867
rebuilt         7140
salvage         3837
lien            1421
missing          797
parts only       189
Name: count, dtype: int64

In [91]:
imputer1 = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
imputer2 = SimpleImputer(missing_values=np.nan,strategy='median')

df1['odometer'] = imputer2.fit_transform(df1[['odometer']])
df1['year'] = imputer1.fit_transform(df1[['year']])




In [92]:
df1['title_status'].fillna(value = df1['title_status'].mode(),inplace=True)

In [93]:
df1.isnull().sum()

price                0
year                 0
odometer             0
title_status      7733
transmission      1823
drive           120254
type             85932
paint_color     117149
state                0
posting_date        62
dtype: int64

In [94]:
df1.shape

(393985, 10)

# EDA

In [95]:
df1.dtypes

price             int64
year            float64
odometer        float64
title_status     object
transmission     object
drive            object
type             object
paint_color      object
state            object
posting_date     object
dtype: object

In [96]:
df1.drop(df1[(df1['price']>1)&(df1['price']<=100)].index,axis =0,inplace=True)

In [110]:
columns1 = ['title_status','transmission','drive','type','paint_color','posting_date']

for col in columns1:
    
    print(df1[col].mode())



0    clean
Name: title_status, dtype: object
0    automatic
Name: transmission, dtype: object
0    4wd
Name: drive, dtype: object
0    sedan
Name: type, dtype: object
0    white
Name: paint_color, dtype: object
0    2021-04-23T22:13:05-0400
Name: posting_date, dtype: object


In [111]:
df1['title_status'] = df1['title_status'].fillna('clean')
df1['transmission'] = df1['transmission'].fillna('automatic')
df1['drive'] = df1['transmission'].fillna('4wd')
df1['type'] = df1['transmission'].fillna('sedan')
df1['paint_color'] = df1['transmission'].fillna('white')
df1['posting_date'] = df1['transmission'].fillna('2021-04-23T22:13:05-0400')


In [112]:
df1.isnull().sum()

price           0
year            0
odometer        0
title_status    0
transmission    0
drive           0
type            0
paint_color     0
state           0
posting_date    0
dtype: int64

In [113]:
sns.histplot(df1['price'])

KeyboardInterrupt: 